In [ ]:
#question 1
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# Download stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Sample documents
doc1 = "Natural language processing helps computers understand human language."
doc2 = "NLP enables machines to process and analyze human language effectively."

# Function for text preprocessing
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    tokens = text.split()  # Tokenization
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Preprocess the documents
doc1_clean = preprocess(doc1)
doc2_clean = preprocess(doc2)

# 1. **Cosine Similarity using TF-IDF**
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([doc1_clean, doc2_clean])
cos_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

# 2. **Jaccard Similarity**
def jaccard_similarity(doc1, doc2):
    set1, set2 = set(doc1.split()), set(doc2.split())
    return len(set1 & set2) / len(set1 | set2)

jaccard_sim = jaccard_similarity(doc1_clean, doc2_clean)

# **Results Comparison**
print(f"Cosine Similarity: {cos_sim:.4f}")
print(f"Jaccard Similarity: {jaccard_sim:.4f}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aspire_Lays\AppData\Roaming\nltk_data...


Cosine Similarity: 0.2090
Jaccard Similarity: 0.1538


[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
#question 2
from textblob import TextBlob
import pandas as pd

# Sample text data
texts = [
    "I absolutely love this product! It works like a charm.",
    "The movie was okay, but it could have been better.",
    "I had a terrible experience with the customer service.",
    "This is the best phone I have ever used!",
    "The weather is neither too hot nor too cold today."
]

# Function to analyze sentiment
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity  # Range: -1 (negative) to +1 (positive)
    subjectivity = blob.sentiment.subjectivity  # Range: 0 (objective) to 1 (subjective)
    return polarity, subjectivity

# Apply sentiment analysis to all texts
results = [analyze_sentiment(text) for text in texts]

# Create a DataFrame for better visualization
df = pd.DataFrame(texts, columns=["Text"])
df["Polarity"] = [r[0] for r in results]
df["Subjectivity"] = [r[1] for r in results]

# Display results
print(df)


                                                Text  Polarity  Subjectivity
0  I absolutely love this product! It works like ...     0.625         0.600
1  The movie was okay, but it could have been bet...     0.500         0.500
2  I had a terrible experience with the customer ...    -1.000         1.000
3           This is the best phone I have ever used!     1.000         0.300
4  The weather is neither too hot nor too cold to...    -0.175         0.925


In [5]:
#question 3
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords

# Download stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aspire_Lays\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Sample dataset (label: 1 = positive, 0 = negative)
# data = {
#     "text": [
#         "I love this movie, it was amazing!",
#         "The food was absolutely terrible.",
#         "I enjoyed the performance a lot.",
#         "The service was very bad, never coming again.",
#         "What a fantastic experience, totally worth it!",
#         "Horrible customer service, very disappointed.",
#         "I am happy with the product.",
#         "This is the worst experience I’ve had.",
#     ],
#     "label": [1, 0, 1, 0, 1, 0, 1, 0],
# }

df_org = pd.read_csv("train.csv",encoding="latin1")



In [31]:
df_org

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
...,...,...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,night,31-45,Ghana,31072940,227540.0,137
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,morning,46-60,Greece,10423054,128900.0,81
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,noon,60-70,Grenada,112523,340.0,331
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,night,70-100,Guatemala,17915568,107160.0,167


In [32]:
df = df_org[["selected_text","sentiment"]]
df

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative
...,...,...
27476,d lost,negative
27477,", don`t force",negative
27478,Yay good for both of you.,positive
27479,But it was worth it ****.,positive


In [33]:
df["sentiment"].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [34]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df["sentiment"] = encoder.fit_transform(df["sentiment"])

C:\Users\Aspire_Lays\AppData\Local\Temp\ipykernel_1696\1458995952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"] = encoder.fit_transform(df["sentiment"])


In [ ]:
df

,selected_text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD,0
2,bullying me,0
3,leave me alone,0
4,"Sons of ****,",0
...,...,...
27476,d lost,0
27477,", don`t force",0
27478,Yay good for both of you.,2
27479,But it was worth it ****.,2


In [39]:
# Function for text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    tokens = text.split()  # Tokenization
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Convert NaN/float values to empty strings before preprocessing
df["selected_text"] = df["selected_text"].astype(str)

# Apply preprocessing
df["clean_text"] = df["selected_text"].apply(preprocess_text)


C:\Users\Aspire_Lays\AppData\Local\Temp\ipykernel_1696\3868434662.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["selected_text"] = df["selected_text"].astype(str)
C:\Users\Aspire_Lays\AppData\Local\Temp\ipykernel_1696\3868434662.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["clean_text"] = df["selected_text"].apply(preprocess_text)


In [41]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["sentiment"], test_size=0.2, random_state=42)

# **Bag of Words (BoW)**
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# **TF-IDF**
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [44]:
# Train Multinomial Naïve Bayes with BoW
nb_bow = MultinomialNB()
nb_bow.fit(X_train_bow, y_train)
y_pred_bow = nb_bow.predict(X_test_bow)

# Train Multinomial Naïve Bayes with TF-IDF
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)

# **Evaluate Models**
print("Multinomial Naïve Bayes (BoW) Performance:")
print(classification_report(y_test, y_pred_bow))
print(accuracy_score(y_test, y_pred_bow))

print("Multinomial Naïve Bayes (TF-IDF) Performance:")
print(classification_report(y_test, y_pred_tfidf))
print(accuracy_score(y_test, y_pred_tfidf))



Multinomial Naïve Bayes (BoW) Performance:
              precision    recall  f1-score   support

           0       0.88      0.62      0.73      1562
           1       0.69      0.90      0.78      2230
           2       0.86      0.76      0.81      1705

    accuracy                           0.78      5497
   macro avg       0.81      0.76      0.77      5497
weighted avg       0.80      0.78      0.78      5497

0.7769692559577952
Multinomial Naïve Bayes (TF-IDF) Performance:
              precision    recall  f1-score   support

           0       0.90      0.60      0.72      1562
           1       0.69      0.93      0.79      2230
           2       0.88      0.75      0.81      1705

    accuracy                           0.78      5497
   macro avg       0.82      0.76      0.77      5497
weighted avg       0.81      0.78      0.78      5497

0.7796980171002365
